In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
url = "C:/Users/mesut/OneDrive/Masaüstü/python/IBB/"
df = pd.read_csv(url+"gunluk-arac-saym.csv", sep = ";", encoding = 'iso8859_9')
df['Tarih'] = pd.to_datetime(df['Tarih'])
df = df.dropna(subset=['X Koordinati'])
df = df.drop(df[df['Sensor Adi'] == 'İsbak Test'].index)

veri_adet = df["Sensor Adi"].value_counts()
mask = df['Sensor Adi'].map(veri_adet) > 10
df = df[mask]
df=df.loc[(df["Arac Sayisi"]>300)]
max_counts = df.groupby(['Tarih', 'Sensor Adi'])['Arac Sayisi'].max().reset_index()
df = df.merge(max_counts, on=['Tarih', 'Sensor Adi', 'Arac Sayisi'], how='inner')
df = df.reset_index(drop=True)

In [2]:
temp = df["Sensor Adi"].unique()
temp2 = df["Sensor Adi"].value_counts()

mask = df['Sensor Adi'].map(temp2) > 10
df = df[mask]

grouped = df.groupby('Sensor Adi')

count = grouped.count()

In [3]:
null_list = [0] * 480

start_date = '2023-01-01'
end_date = '2023-12-31'
num_values = 480

random_dates = pd.to_datetime(np.random.randint(pd.Timestamp(start_date).timestamp(), 
                                                pd.Timestamp(end_date).timestamp(), 
                                                num_values), 
                              unit='s')

new_columns = {
    'start_date': random_dates,
    'end_date': random_dates,
    'total_date_number': null_list,
    'missing_date_number': null_list,   
    'missing_value_rate': null_list,
    'missing_date_list': [0] * 480
    
}
count = count.assign(**new_columns)
max_counts = df.groupby(['Tarih', 'Sensor Adi'])

for name, group in grouped:
    start_date = group['Tarih'].min()
    end_date = group['Tarih'].max()
    full_date_range = pd.date_range(start=start_date, end=end_date)
    missing_dates = full_date_range[~full_date_range.isin(group['Tarih'])]
    columns_to_change = ['start_date', 'end_date', 'total_date_number', 'missing_date_number', 'missing_value_rate', 'missing_date_list']
    new_values = [start_date, end_date, len(full_date_range), len(missing_dates), len(missing_dates)/len(full_date_range), list(missing_dates)]
    count.loc[name, columns_to_change] = new_values
 

In [4]:
count = count.loc[(count['Tarih'] > 1500) & (count['missing_value_rate'] < 0.1)
                  & (count['missing_date_number'] < 200)]
df = df[df['Sensor Adi'].isin(list(count.index))]
df.shape

(265365, 5)

In [5]:
def ifnan(date, df, sayac = 0):
    if date in df.index:

        # if #dataframe'in sonuna ya da başına gelme durumunu kontrol et
        #sonuna ya da başına geldiysen tüm haftalarda o günün ortalamasını al.
        if pd.isna(df.loc[date]['Arac Sayisi']) == True: # o günün verisi var mı
            #print('o gün yok')
            new_prev_week = date - pd.DateOffset(weeks=1) # bir önceki ya da sonraki haftaya geç
            return ifnan(new_prev_week, df) # fonksiyonu tekrar çağır
        else:

            return df.loc[date]['Arac Sayisi']
    else: # df'nin başına ya da sonuna geldil
        weekday = date.weekday() # haftanın hangi günü
        same_weekday_data = df[df.index.weekday == weekday]['Arac Sayisi']
        # tüm df'de o günün değerlerinin ortalamasını al
        avg = same_weekday_data.mean()
        return avg
    
def ifnan2(date, df, sayac = 0):
    if date in df.index:
        # if #dataframe'in sonuna ya da başına gelme durumunu kontrol et
        #sonuna ya da başına geldiysen tüm haftalarda o günün ortalamasını al.
        if pd.isna(df.loc[date]['Arac Sayisi']) == True: # o günün verisi var mı
            #print('o gün yok')
            new_prev_week = date + pd.DateOffset(weeks=1) # bir önceki ya da sonraki haftaya geç
            return ifnan(new_prev_week, df) # fonksiyonu tekrar çağır
        else:
            return df.loc[date]['Arac Sayisi']
    else: # df'nin başına ya da sonuna geldil
        weekday = date.weekday() # haftanın hangi günü
        same_weekday_data = df[df.index.weekday == weekday]['Arac Sayisi']
        # tüm df'de o günün değerlerinin ortalamasını al
        avg = same_weekday_data.mean()
        return avg

In [6]:
sensors = [df['Sensor Adi'].unique()[0]]
def eksik(sensor, df):
    
    temmuz=df[df["Sensor Adi"]== sensor]
    start_date = temmuz['Tarih'].min()
    end_date = temmuz['Tarih'].max()
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    missing_dates = full_date_range.difference(temmuz['Tarih'])
    missing_df = pd.DataFrame({'Tarih': missing_dates})
    missing_df['Sensor Adi'] = temmuz['Sensor Adi'].iloc[0]  
    missing_df['X Koordinati'] = temmuz['X Koordinati'].iloc[0] 
    missing_df['Y Koordinati'] = temmuz['Y Koordinati'].iloc[0]
    df = pd.concat([temmuz, missing_df])
    df.set_index('Tarih', inplace=True)
    for missing_date in missing_dates:
        prev_week = missing_date - pd.DateOffset(weeks=1)
        next_week = missing_date + pd.DateOffset(weeks=1)

        # Önceki haftanın ve sonraki haftanın "Arac Sayisi" değerlerini alın
        prev_week_data = ifnan(prev_week, df)
        next_week_data = ifnan2(next_week, df)
        #prev_week_data = df.loc[prev_week]['Arac Sayisi']
        #next_week_data = df.loc[next_week]['Arac Sayisi']
        # print(prev_week , prev_week_data)
        # print(prev_week_data, next_week_data)
        # Ortalamayı hesaplayın ve eksik güne ekleyin
        avg = int((prev_week_data + next_week_data) / 2)
        df.at[missing_date, 'Arac Sayisi'] = avg
        # df.reset_index(inplace=True)
        #break
    return df

#deneme = eksik(sensors, df)

In [7]:
dff = pd.DataFrame()
#sensors = [df['Sensor Adi'].unique()
for sensor in df['Sensor Adi'].unique():
    deneme = eksik(sensor, df)
    dff = pd.concat([dff, deneme])

else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi
else girdi


In [8]:
dff.shape 

(276456, 4)

In [9]:
dff.sort_index().head(10)

,Sensor Adi,X Koordinati,Y Koordinati,Arac Sayisi
Tarih,,,,
2016-01-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",49912.0
2016-01-01,TEM Orhanlı,"29,36342733","40,90512054",33102.0
2016-01-01,D100 Kartal Kavşağı,"29,21172042","40,905516",57777.0
2016-01-01,Havaalanı İski Önü,"28,83919773","40,97660915",23434.0
2016-01-01,Alibeyköy TEM Katılımı,"28,95829669","41,07221143",3241.0
2016-01-01,TEM Ataşehir Şerifali,"29,13212132","40,99691649",82262.0
2016-01-01,TEM Olimpiyat Stadı,"28,75357025","41,0611654",54903.0
2016-01-01,D100 Kartal Oto Sanayi,"29,19466674","40,91127817",53779.0
2016-01-01,Havaalanı Florya,"28,80277776","40,98015352",10397.0


In [10]:
dff.tail(10)

,Sensor Adi,X Koordinati,Y Koordinati,Arac Sayisi
Tarih,,,,
2020-11-22,Beylikdüzü,"28,638692","41,016302",102485.0
2020-11-23,Beylikdüzü,"28,638692","41,016302",123353.0
2020-11-24,Beylikdüzü,"28,638692","41,016302",107417.0
2020-11-25,Beylikdüzü,"28,638692","41,016302",112696.0
2020-11-26,Beylikdüzü,"28,638692","41,016302",114667.0
2020-11-27,Beylikdüzü,"28,638692","41,016302",113801.0
2020-11-28,Beylikdüzü,"28,638692","41,016302",113271.0
2020-11-29,Beylikdüzü,"28,638692","41,016302",118584.0
2020-11-30,Beylikdüzü,"28,638692","41,016302",128966.0


In [11]:
nan_sayisi = deneme['Arac Sayisi'].isna().sum()
nan_sayisi

0

In [12]:
print(dff.loc['2020-12-4'])

                                            Sensor Adi X Koordinati  \
Tarih                                                                 
2020-12-04  15 Temmuz Şehitler Köprüsü Yıldız Katılımı  29,01843233   
2020-12-04                                    Akom Önü    28,961073   
2020-12-04                      Alibeyköy TEM Katılımı  28,95829669   
2020-12-04                                  Altunizade  29,04612265   
2020-12-04                Altunizade Ümraniye Katılımı  29,04965362   
...                                                ...          ...   
2020-12-04                                 D100 Unalan  29,05949306   
2020-12-04                              Memorial Hast.   28,9771321   
2020-12-04                               Okmeydanı DMS    28,949946   
2020-12-04                     Güneşli B. Ekspres Yolu  28,80777032   
2020-12-04                                  Beylikdüzü    28,638692   

           Y Koordinati  Arac Sayisi  
Tarih                                

In [13]:
#deneme['HaftaninGunu'] = deneme.index.dayofweek
#gruplanmis_df = deneme.groupby('HaftaninGunu')['Arac Sayisi'].mean()
#gruplanmis_df

In [14]:
#df['HaftaninGunu'] = df['Tarih'].dt.dayofweek
#gruplanmis_df = df.groupby('HaftaninGunu')['Arac Sayisi'].mean()
#gruplanmis_df

In [15]:
dff.head(10)

,Sensor Adi,X Koordinati,Y Koordinati,Arac Sayisi
Tarih,,,,
2016-01-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",49912.0
2016-02-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",58038.0
2016-03-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",63710.0
2016-04-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",74919.0
2016-05-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",78458.0
2016-06-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",80988.0
2016-07-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",77371.0
2016-08-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",83001.0
2016-09-01,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",82439.0


In [16]:
dff['Tarih'] = dff.index
dff.reset_index(drop=True, inplace=True)
dff

,Sensor Adi,X Koordinati,Y Koordinati,Arac Sayisi,Tarih
0,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",49912.0,2016-01-01
1,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",58038.0,2016-02-01
2,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",63710.0,2016-03-01
3,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",74919.0,2016-04-01
4,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,"29,01843233","41,05770224",78458.0,2016-05-01
...,...,...,...,...,...
276451,Beylikdüzü,"28,638692","41,016302",113801.0,2020-11-27
276452,Beylikdüzü,"28,638692","41,016302",113271.0,2020-11-28
276453,Beylikdüzü,"28,638692","41,016302",118584.0,2020-11-29
276454,Beylikdüzü,"28,638692","41,016302",128966.0,2020-11-30


In [17]:
dff.to_csv(url+'veriler5.csv', index=False)  # index=False, indeksi CSV dosyasına dahil etmez
